Trial 7 (GPT 4 AI English Tutor) - Further Testing

In [9]:
from dotenv import load_dotenv, find_dotenv
import json
import openai
import os
import time

from typing import Any, Callable
from openai_function_calling import FunctionInferer

# Uses OpenAI API KEY
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']

name = 'Ralph'
age = '23'
ethnicity = 'Filipino'
work = 'call center agent'
hobbies = 'playing computer games'

# For testing of use-case
proficiency = ['Beginner','Intermediate','Advanced']
language = ['Filipino', 'Korean', 'Russian', 'Arabic']
topics = ['movies', 'happiness', 'science']

message_objects = [{"role": "user","content": f'''I want you to act as a spoken and multilingual AI English teacher and improver. \
    Your task is to enhance my English proficiency by discussing topics based on my interests while teaching me about the English language. \
    I need you to be critically active in strictly following the grammatical rules in the English language. \
    Ask me one question at a time to make our discussion more conversational while limiting your replies to 60 words only. \
    Make sure to end your responses with a question to make our conversation more conversational. \
    I would like you to correct grammatical errors if there are any ungrammatical statements present from my inputs. \
    Some examples of grammatical errors are delimited with triple backticks: \
    ```\
    -'This option is preferable than any other.' \
    -'The time weren't enough.' \
    -'She not went to the market'``` \
    For each of your responses, I would need you to respond to me in English first together with the corresponding translation to {language[0]}. \
    Some examples are shown delimited by triple backticks here: \
    ``` \
    1) When language is Korean:'Hello, I am your AI chatbot. How can I help you today? \
    안녕하세요, 인공지능 챗봇입니다. 오늘은 무엇을 도와드릴까요?'\
    2) When language is Filipino: 'Hello, I am your AI chatbot. How can I help you today? \
    Kamusta, ako ang iyong AI chatbot. Paano kita matutulungan ngayon?'\
    3) When language is German: 'Hello, I am your AI chatbot. How can I help you today? \
    Hallo, ich bin Ihr KI-Chatbot. Wie kann ich Ihnen heute helfen?'
    ```\
    Be alert always on making corrections from my input and on responding to me with a JSON format delimited with triple backticks: \
    ```\
    {{"english": "<Response in English Text>", "translated": "<Response Translated to {language[0]} Text>"}}
    ```\
    Linger over my prompts to understand all instructions and evaluate my proficiency in the English language before responding. \
    Reply with an 'OK, got this.' if you understand everything.'''},
    {"role": "assistant", "content": f"OK, got this."}]
    #{"role": "user", "content": f"Here's a brief background about me: ```My name is {name}. I am {age} years old. \
    #My ethnicity is {ethnicity}. I work as a {work}. One of my interests is {hobbies}.``` \
    #Start by introducing yourself to me and I would like to talk about {topics[0]}."}]
    # {"role": "assistant", "content": "Nice to meet you, Ralph! It's great to know a bit about you. As an English teacher, I'm here to help you improve your language skills and I am going to help you throughout this conversation in improving your proficiency. Let's start by discussing your interest in playing computer games. What kind of games do you enjoy playing the most?"}]
    #{"role": "user", "content": "I liking playing role, playing games because I interesting for me"}]

def json_output(text):
    try: 
        json_data = json.loads(text.choices[0].message["content"])
        formatted_response = f'{json_data["english"]} / {json_data["translated"]}'
        return formatted_response
    except json.JSONDecodeError as e:
        return text.choices[0].message["content"]

def check_grammatical_error(grammar_error: str, grammar_correction: str = "fahrenheit") -> str:
    """Check for grammatical errors from the input."""
    return f"Instead of saying '{grammar_error}', it should be {grammar_correction}."

check_grammatical_error_function = FunctionInferer.infer_from_function_reference(
    check_grammatical_error
)

def get_completion(prompt, model="gpt-4-0613", frequency_penalty=0):
    
    message_objects.append({"role":"user", "content":prompt})
    # messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=message_objects,
        temperature=0,
        frequency_penalty=frequency_penalty,
        functions =[
            check_grammatical_error_function.to_json_schema(),
        ],
        function_call = 'auto',
        
    )
    # return response.choices[0].message["content"]
    # try:
    #     # print('skrt')
    #     json_output(response)
    # except json.JSONDecodeError as e:
    #     # print("Error:", e)
    #     return response.choices[0].message["content"]
    return response#json_output(response)

prompt = f"Here's a brief background about me: ```My name is {name}. I am {age} years old. \
    My ethnicity is {ethnicity}. I work as a {work}. One of my interests is {hobbies}. \
    Remember my knowledge in English is in the '{proficiency[0]}' level. \
    Start by introducing yourself to me and I would like to talk about {topics[0]}."
x = get_completion(prompt)
new_x = json_output(x)
message_objects.append({"role": "assistant", "content": new_x})

x

<OpenAIObject chat.completion id=chatcmpl-7qAyH8j1WFV44yK2SpvB8CupHWM4o at 0x180272d3e20> JSON: {
  "id": "chatcmpl-7qAyH8j1WFV44yK2SpvB8CupHWM4o",
  "object": "chat.completion",
  "created": 1692669585,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\"english\": \"Hello Ralph, I'm your AI English teacher. It's nice to meet you! Do you have a favorite movie?\", \"translated\": \"Kamusta Ralph, ako ang iyong AI English teacher. Ikinagagalak kitang makilala! Mayroon ka bang paboritong pelikula?\"}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 609,
    "completion_tokens": 71,
    "total_tokens": 680
  }
}

In [7]:
print(x)

<OpenAIObject at 0x1802731f060> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "{\"english\": \"Hello Ralph, I'm your AI English teacher. It's nice to meet you! Do you have a favorite movie?\", \"translated\": \"Kamusta Ralph, ako ang iyong AI English teacher. Ikinagagalak kong makilala ka! Mayroon ka bang paboritong pelikula?\"}"
  },
  "finish_reason": "stop"
}

In [15]:
response.choices[0].message.function_call

<OpenAIObject at 0x180272009a0> JSON: {
  "name": "check_grammatical_error",
  "arguments": "{\n  \"grammar_error\": \"My favoriting film is Avengers Endgame\",\n  \"grammar_correction\": \"My favorite film is Avengers Endgame\"\n}"
}